In [1]:
#!git clone https://github.com/wesleymsmith/Piezo_PIP2_binding_analysis.git
#!pip install bokeh
import numpy as np
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import scipy as sp
from scipy import stats

import ipywidgets as widgets
from ipywidgets import interact, interact_manual

import os
import sys
import gc
import copy
import glob

import tqdm
import itertools

import bokeh
from bokeh.layouts import gridplot
from bokeh.models import ColumnDataSource, CDSView, GroupFilter, HoverTool
from bokeh.plotting import figure, show
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6

# Load Occupancy Data

In [6]:
baseDir='.'

aaOccupancyFile="All_Atom_Occupancy_2uS_Data.csv"
aaFrameRate=.6

print 'reading all atom data'
aa_occupancy_data=pd.read_csv('/'.join([baseDir,aaOccupancyFile]))
aa_occupancy_data.Time=aa_occupancy_data.Frame*aaFrameRate
aa_occupancy_data['SimType']='AA'

print aa_occupancy_data.head()

cgOccupancyFileBase="Coarse_Grain_Occupancy_Data.chunk"
cgFrameRate=1.0

cgOccupancyFileList=[fileName for fileName in \
                     os.listdir(baseDir) if \
                     cgOccupancyFileBase in fileName]
#print cgOccupancyFileList
cgOccupancyChunks=[]
print 'Reading coarse grain data chunks'
cgColumns=[]
for iFile,fileName in tqdm.tqdm_notebook(enumerate(cgOccupancyFileList)):
    if 'chunk.aa' in fileName:
        cgOccupancyChunks.append(pd.read_csv('/'.join([baseDir,fileName])))
        print "cg occupancy file header:"
        cgColumns=cgOccupancyChunks[iFile].columns
        print cgColumns
    else:
        cgOccupancyChunks.append(pd.read_csv('/'.join([baseDir,fileName]),
            header=None))
for cgFrame in cgOccupancyChunks:
    cgFrame.columns=cgColumns
cg_occupancy_data=pd.concat(cgOccupancyChunks)
del(cgOccupancyChunks)
cg_occupancy_data.Time=cg_occupancy_data.Frame*cgFrameRate
cg_occupancy_data['Rep']=0
cg_occupancy_data['SimType']='CG'
print cg_occupancy_data.head()
gc.collect()

joint_occupancy_data=pd.concat([aa_occupancy_data,cg_occupancy_data])
joint_occupancy_data.head()

reading all atom data
   ResID  SeqID  Frame  Time  Occupancy  Rep SimType
0   3877   2169      0   0.0          0    0      AA
1   3877   2169      1   0.6          0    0      AA
2   3877   2169      2   1.2          0    0      AA
3   3877   2169      3   1.8          0    0      AA
4   3877   2169      4   2.4          0    0      AA
Reading coarse grain data chunks


cg occupancy file header:
Index([u'ResID', u'SeqID', u'Frame', u'Time', u'Occupancy'], dtype='object')

   ResID  SeqID  Frame    Time  Occupancy  Rep SimType
0    589   1497   3866  3866.0          0    0      CG
1    589   1497   3867  3867.0          1    0      CG
2    589   1497   3868  3868.0          1    0      CG
3    589   1497   3869  3869.0          1    0      CG
4    589   1497   3870  3870.0          1    0      CG


,ResID,SeqID,Frame,Time,Occupancy,Rep,SimType
0,3877,2169,0,0.0,0,0,AA
1,3877,2169,1,0.6,0,0,AA
2,3877,2169,2,1.2,0,0,AA
3,3877,2169,3,1.8,0,0,AA
4,3877,2169,4,2.4,0,0,AA


In [49]:
groupingCols=['ResID','SeqID','Rep','SimType']
with pd.option_context('mode.use_inf_as_null',True):
    analysisData=joint_occupancy_data
    analysisData['CumulativeOccupancy']=analysisData.groupby(groupingCols).Occupancy.cumsum()
    analysisData['CumulativeMean']=aaFrameRate*analysisData.CumulativeOccupancy/analysisData.Frame
analysisData.head()

,ResID,SeqID,Frame,Time,Occupancy,Rep,SimType,CumulativeOccupancy,CumulativeMean
0,3877,2169,0,0.0,0,0,AA,0,NaN
1,3877,2169,1,0.6,0,0,AA,0,0.0
2,3877,2169,2,1.2,0,0,AA,0,0.0
3,3877,2169,3,1.8,0,0,AA,0,0.0
4,3877,2169,4,2.4,0,0,AA,0,0.0


In [61]:
@interact
def plot_cumMeans(resid=analysisData.ResID.unique(),
                  simType=analysisData.SimType.unique()):
    tempData=analysisData[(analysisData.ResID==resid) & \
                          (analysisData.SimType==simType)]
    for rep in tempData.Rep.unique():
        plotData=tempData[tempData.Rep==rep]
        plt.plot(plotData.Time,plotData.CumulativeMean,
                 label=rep)
    plt.legend()
    plt.show()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KERyb3Bkb3duKGRlc2NyaXB0aW9uPXUncmVzaWQnLCBvcHRpb25zPSgzODc3LCAzNzMsIDE1NDAsIDMzOTAsIDIyMTIsIDM2NDksIDc2MCwgNDA1OSwgMTXigKY=
